In [ ]:
import geopandas as gpd, warnings,pandas as pd, folium
from folium.plugins import Search
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
%matplotlib inline
pd.set_option('display.float_format', '{:.2f}'.format)

In [ ]:
subdivision_new_file = gpd.read_file("../../../../../../Edited Subdivisions Zurikanen Erfan 4-20=2025/subs_plus_new/sub.gdb",layer='final_10_9_25')
subdivision_new_file["issue_date"] = subdivision_new_file["issue_date"].dt.year.astype(str)

In [ ]:
#Finding overlapped subs
sindex = subdivision_new_file.sindex

overlapping_indices = set() # Set to hold overlapping polygon indices


# Loop to check overlap area
for idx, geom in subdivision_new_file.geometry.items():
    possible_matches_index = list(sindex.intersection(geom.bounds))     # Get candidate matches via bounding box  
    for i in possible_matches_index:
        if i != idx:
            other_geom = subdivision_new_file.geometry.iloc[i]
            if geom.intersects(other_geom):
                intersection = geom.intersection(other_geom)
                if intersection.area >= 21000:  # 1 acre in square feet in espg 2264
                    overlapping_indices.add(idx)
                    break  # Found one overlap ≥ 1 acre; no need to check more
                    
overlapping_subdivisions = subdivision_new_file.loc[list(overlapping_indices)]

In [ ]:
#reproject to global crs recognized by folium
subdivision_new_file = subdivision_new_file.to_crs(epsg=4326)
overlapping_subdivisions = overlapping_subdivisions.to_crs(epsg=4326)

In [ ]:
map = folium.Map(
    location=(35.22648448735279, -80.84093065828473),
    zoom_start=13,
    tiles='OpenStreetMap'
)

In [ ]:
map_overalpped = folium.Map(
    location=(35.22648448735279, -80.84093065828473),
    zoom_start=13,
    tiles='OpenStreetMap'
)

In [ ]:
popup = folium.GeoJsonPopup(
    fields=["subd_id", 'issue_date'],
    localize=True,
    labels=True,
    style="background-color: yellow;"
)

In [ ]:
subdivision_new_file_json = subdivision_new_file[['subd_id','year_1990','issue_date','geometry']].to_json()
overlapping_subdivisions_json = overlapping_subdivisions[['subd_id','year_1990','issue_date','geometry']].to_json()

In [ ]:
subs_new = folium.GeoJson(
    data=subdivision_new_file_json,
    name="Subdivisions",
    style_function=lambda feature: {
        "color": "green" if feature["properties"]["year_1990"] == 1 else "red",
        "weight": 1,
        "fillOpacity": 0.3,
    },
    highlight_function=lambda feature: {
        "color": "black",  # Darker boundary on hover/click
        "weight": 4,
        "fillOpacity": 0.6
    },
    popup=popup
)
subs_new.add_to(map)

search = Search(
    layer=subs_new,
    search_label="subd_id",
    placeholder="Search by Sub ID",
    collapsed=False
)
search.add_to(map)

In [ ]:
# Define the style function
def style_function(feature):
    return {
        'fillColor': '#3388ff',
        'color': 'Blue',
        'weight': 1.5,
        'fillOpacity': 0.5
    }

subs_overlapped = folium.GeoJson(
    data = overlapping_subdivisions_json,
    name="Subdivisions",
    style_function=style_function,
    highlight_function=lambda feature: {
    "color": "black",
    "weight": 4,
    "fillOpacity": 0.6
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["subd_id",'issue_date'],
        sticky=True
    )
)
subs_overlapped.add_to(map_overalpped)

search = Search(
    layer=subs_overlapped,
    search_label="subd_id",
    placeholder="Search by Sub ID",
    collapsed=False
)
search.add_to(map_overalpped)

In [14]:
map.save("../../../../../../Edited Subdivisions Zurikanen Erfan 4-20=2025/subdivisions32.html")
map_overalpped.save("../../../../../../Edited Subdivisions Zurikanen Erfan 4-20=2025/subdivisions_overlapped.html")